# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [31]:
!pip install pathway bokeh --quiet

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime , timedelta
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [48]:
df = pd.read_csv('/content/dataset.csv')
df


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [49]:

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

In [50]:
print(df.columns.tolist())


['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'Timestamp']


In [51]:
selected_columns = [
    "Timestamp",
    "lot_id",
    "Occupancy",
    "Capacity",
    "QueueLength",
    "VehicleType",
    "TrafficConditionNearby",
    "IsSpecialDay"
]
df.rename(columns={"SystemCodeNumber": "lot_id"}, inplace=True)


df[selected_columns].to_csv("parking_stream.csv", index=False)


In [52]:

import pathway as pw

class ParkingSchema(pw.Schema):
    Timestamp: str
    lot_id: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    VehicleType: str
    TrafficConditionNearby: str
    IsSpecialDay: int


In [53]:


data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [54]:

fmt = "%Y-%m-%d %H:%M:%S"


data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),

)



In [40]:
import pathway as pw
print(pw.__version__)


0.24.1


# Step 2: Making a simple pricing function

In [41]:
## model 1 delta function
base_price = 10.0
alpha = 0.5


delta_window = (
    data_with_time
    .windowby(
        pw.this.t,
        instance=pw.this.lot_id + "_" + pw.this.day,
        window=pw.temporal.tumbling(timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ=pw.reducers.max(pw.this.Occupancy),   # used max instead of last
        cap=pw.reducers.max(pw.this.Capacity),    # max or any – both ok
        lot_id=pw.reducers.any(pw.this.lot_id)    # to retain lot info
    )
    .with_columns(
        price = base_price + alpha * (pw.this.occ / pw.this.cap)
    )
)


# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [42]:

pn.extension()


def price_plotter(source):

    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="navy")


    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")

pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [47]:

%%capture --no-display
pw.run()

Output()

In [55]:
import datetime
import pathway as pw

base_price = 10.0
queue_weight = 0.5
special_day_bonus = 2.0

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.lot_id),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        sum_queue=pw.reducers.sum(pw.this.QueueLength),
        count=pw.reducers.count(),
        traffic_sample=pw.reducers.any(pw.this.TrafficConditionNearby),
        vehicle_sample=pw.reducers.any(pw.this.VehicleType),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        lot_id=pw.reducers.any(pw.this.lot_id),
    )
    .with_columns(
        avg_queue = pw.this.sum_queue / pw.this.count,
        demand_fluctuation = (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
    .with_columns(
        price = base_price
            + pw.this.demand_fluctuation
            + queue_weight * pw.this.avg_queue / 10.0
            + pw.this.is_special_day * special_day_bonus
    )
)


In [56]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price using model 2",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")

pn.Row(viz).servable()

Row
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [57]:


%%capture --no-display
pw.run()

Output()